# OpenAI Function Calling

Based on [**this tutorial**](https://learn.deeplearning.ai/courses/functions-tools-agents-langchain/lesson/2/openai-function-calling)

# Setup

In [1]:
from dotenv import load_dotenv

In [2]:
_ = load_dotenv()

# Global Imports

In [3]:
import json

import openai
from rich import print as rprint

# Code

We'll implement a dummy function to get the current weather.

This is an interesting example, as LLMs are naturally not able to perform this.

Hence, **we will often want to connect LLMs to this kind of function to augment them**.

## Implement and Define Functions

In [4]:
# Example dummy function hard coded to return the same weather
# In production, this could be your backend API or an external API
def get_current_weather(location, unit="fahrenheit"):
    """Get the current weather in a given location"""
    weather_info = {
        "location": location,
        "temperature": "72",
        "unit": unit,
        "forecast": ["sunny", "windy"],
    }
    return json.dumps(weather_info)

In [5]:
get_current_weather("Valenciennes")

'{"location": "Valenciennes", "temperature": "72", "unit": "fahrenheit", "forecast": ["sunny", "windy"]}'

So, **how do we pass this information to the model?**

OpenAI has exposed a new parameter called `functions` through which you can pass a list of **function definitions**.

Here's the full **function definition** for our previous function, which is a JSON object with a few fields.

In [6]:
# Define a function:
functions = [
    {
        "name": "get_current_weather",
        "description": "Get the current weather in a given location",
        "parameters": {
            "type": "object",
            "properties": {
                "location": {
                    "type": "string",
                    "description": "The city and state, e.g. San Francisco, CA",
                },
                "unit": {
                    "type": "string",
                    "enum": ["celsius", "fahrenheit"]
                },
            },
            "required": ["location"],
        },
    }
]

**The `description` fields are really important because the LLM will use these descriptions to determine wether to use this function**.

> **Therefore, any information you want to pass to the LLM to help him determine when and how to use a function should be included inside `"description"` fields**.

## LLM Call

## With a Message Related to the Function

In [7]:
messages = [
    {
        "role": "user",
        "content": "What's the weather like in Boston?"
    }
]

In [8]:
def get_completion(messages: str):
    """Convenience function to perform chat completions"""
    return  openai.chat.completions.create(
        model = "gpt-3.5-turbo-0613",
        messages = messages,
        functions = functions
    )

In [9]:
response =get_completion(messages)

In [10]:
rprint(response)

ChatCompletion(
    id='chatcmpl-9XSQrtbdnVQb4RcoBfeBtVgkkYYXb',
    choices=[
        Choice(
            finish_reason='function_call',
            index=0,
            logprobs=None,
            message=ChatCompletionMessage(
                content=None,
                role='assistant',
                function_call=FunctionCall(arguments='{\n  "location": "Boston"\n}', name='get_current_weather'),
                tool_calls=None
            )
        )
    ],
    created=1717761145,
    model='gpt-3.5-turbo-0613',
    object='chat.completion',
    system_fingerprint=None,
    usage=CompletionUsage(completion_tokens=16, prompt_tokens=82, total_tokens=98)
)

In [11]:
response_message = response.choices[0].message

In [12]:
rprint(response_message)

ChatCompletionMessage(
    content=None,
    role='assistant',
    function_call=FunctionCall(arguments='{\n  "location": "Boston"\n}', name='get_current_weather'),
    tool_calls=None
)

Especially, notice the `function_call` argument.

In [13]:
response_message.function_call

FunctionCall(arguments='{\n  "location": "Boston"\n}', name='get_current_weather')

In [14]:
response_message.function_call.arguments

'{\n  "location": "Boston"\n}'

It's a JSON string. So, we can make use of `json.loads` to load it within a Python dictionary.

In [15]:
json.loads(response_message.function_call.arguments)

{'location': 'Boston'}

In [16]:
args = json.loads(response_message.function_call.arguments)
get_current_weather(args)

'{"location": {"location": "Boston"}, "temperature": "72", "unit": "fahrenheit", "forecast": ["sunny", "windy"]}'

## With a Message Unrelated to the Function

In [17]:
messages = [
    {
        "role": "user",
        "content": "hi!",
    }
]

In [18]:
response = get_completion(messages)

In [19]:
rprint(response)

ChatCompletion(
    id='chatcmpl-9XSQswvjLAq8MCgpSpMs8SjIRVI1Q',
    choices=[
        Choice(
            finish_reason='stop',
            index=0,
            logprobs=None,
            message=ChatCompletionMessage(
                content='Hello! How can I assist you today?',
                role='assistant',
                function_call=None,
                tool_calls=None
            )
        )
    ],
    created=1717761146,
    model='gpt-3.5-turbo-0613',
    object='chat.completion',
    system_fingerprint=None,
    usage=CompletionUsage(completion_tokens=10, prompt_tokens=76, total_tokens=86)
)

What happens, under the hood, is that the model is decided whether to use a function or not.

This explains: `function_call=None`

There exists a `function_call` parameter to force, or not, the model to use a function.

By default, it's set to `"auto"`, letting the model decide if that's relevant or not.

The other modes are:
- `"none"`

In [20]:
response = openai.chat.completions.create(
    model = "gpt-3.5-turbo-0613",
    messages = messages,
    functions = functions,
    function_call = "auto"
)
rprint(response)

ChatCompletion(
    id='chatcmpl-9XSQs7xaxmz8nOUACpfsy43Qn9dz6',
    choices=[
        Choice(
            finish_reason='stop',
            index=0,
            logprobs=None,
            message=ChatCompletionMessage(
                content='Hello! How can I assist you today?',
                role='assistant',
                function_call=None,
                tool_calls=None
            )
        )
    ],
    created=1717761146,
    model='gpt-3.5-turbo-0613',
    object='chat.completion',
    system_fingerprint=None,
    usage=CompletionUsage(completion_tokens=10, prompt_tokens=76, total_tokens=86)
)

## Relevant Message But `function_call="none"`

In [21]:
messages = [
    {
        "role": "user",
        "content": "What's the weather in Boston?",
    }
]
response = openai.chat.completions.create(
    model = "gpt-3.5-turbo-0613",
    messages = messages,
    functions = functions,
    function_call = "none"
)
rprint(response)

ChatCompletion(
    id='chatcmpl-9XSQtMe0oVL4UlW4mnOZGkOZEHbQi',
    choices=[
        Choice(
            finish_reason='stop',
            index=0,
            logprobs=None,
            message=ChatCompletionMessage(
                content='{\n  "location": "Boston, MA"\n}',
                role='assistant',
                function_call=None,
                tool_calls=None
            )
        )
    ],
    created=1717761147,
    model='gpt-3.5-turbo-0613',
    object='chat.completion',
    system_fingerprint=None,
    usage=CompletionUsage(completion_tokens=11, prompt_tokens=82, total_tokens=93)
)

> 😱We should explore why we got the previous response, hypothesis being it hallucinates the response...

## Forcing a Function Call

### With Relevant Messages

In [22]:
messages = [
    {
        "role": "user",
        "content": "What's the weather in Boston?",
    }
]
response = openai.chat.completions.create(
    model = "gpt-3.5-turbo-0613",
    messages = messages,
    functions = functions,
    function_call = {"name": "get_current_weather"}
)
rprint(response)

ChatCompletion(
    id='chatcmpl-9XSQuvER2la55JoeAITRZUMWnKE3F',
    choices=[
        Choice(
            finish_reason='stop',
            index=0,
            logprobs=None,
            message=ChatCompletionMessage(
                content=None,
                role='assistant',
                function_call=FunctionCall(
                    arguments='{\n  "location": "Boston, MA"\n}',
                    name='get_current_weather'
                ),
                tool_calls=None
            )
        )
    ],
    created=1717761148,
    model='gpt-3.5-turbo-0613',
    object='chat.completion',
    system_fingerprint=None,
    usage=CompletionUsage(completion_tokens=11, prompt_tokens=88, total_tokens=99)
)

### With Non-Relevant Messages

In [23]:
messages = [
    {
        "role": "user",
        "content": "Hi!",
    }
]
response = openai.chat.completions.create(
    model = "gpt-3.5-turbo-0613",
    messages = messages,
    functions = functions,
    function_call = {"name": "get_current_weather"}
)
rprint(response)

ChatCompletion(
    id='chatcmpl-9XSQutdQX5tVTCjSp1gOGtpBTwQ4A',
    choices=[
        Choice(
            finish_reason='stop',
            index=0,
            logprobs=None,
            message=ChatCompletionMessage(
                content=None,
                role='assistant',
                function_call=FunctionCall(
                    arguments='{\n  "location": "San Francisco, CA"\n}',
                    name='get_current_weather'
                ),
                tool_calls=None
            )
        )
    ],
    created=1717761148,
    model='gpt-3.5-turbo-0613',
    object='chat.completion',
    system_fingerprint=None,
    usage=CompletionUsage(completion_tokens=12, prompt_tokens=83, total_tokens=95)
)

Here, the LLM is confused and calls the function with its San Francisco, CA...

What happens if running it again?

In [24]:
messages = [
    {
        "role": "user",
        "content": "Hi!",
    }
]
response = openai.chat.completions.create(
    model = "gpt-3.5-turbo-0613",
    messages = messages,
    functions = functions,
    function_call = {"name": "get_current_weather"}
)
rprint(response)

ChatCompletion(
    id='chatcmpl-9XSQvDvRd7kjyzNExUPHtK61ZcJvn',
    choices=[
        Choice(
            finish_reason='stop',
            index=0,
            logprobs=None,
            message=ChatCompletionMessage(
                content=None,
                role='assistant',
                function_call=FunctionCall(
                    arguments='{\n  "location": "San Francisco, CA"\n}',
                    name='get_current_weather'
                ),
                tool_calls=None
            )
        )
    ],
    created=1717761149,
    model='gpt-3.5-turbo-0613',
    object='chat.completion',
    system_fingerprint=None,
    usage=CompletionUsage(completion_tokens=12, prompt_tokens=83, total_tokens=95)
)

It seems to be running it again and again...

# Worth Noting...

The functions and their description count against the token usage limit that you pass to OpenAI.

Here are on runs that will illustrate it if you focus on the `"prompt_token"` counts, as I commented `functions` and `function_calls`...

In [25]:
messages = [
    {
        "role": "user",
        "content": "Hi!",
    }
]
response = openai.chat.completions.create(
    model = "gpt-3.5-turbo-0613",
    messages = messages,
    # functions = functions,
    # function_call = {"name": "get_current_weather"}
)
rprint(response)

ChatCompletion(
    id='chatcmpl-9XSQwwHYPI8zsPOpLTqvnlHeo5Ezx',
    choices=[
        Choice(
            finish_reason='stop',
            index=0,
            logprobs=None,
            message=ChatCompletionMessage(
                content='Hello! How can I assist you today?',
                role='assistant',
                function_call=None,
                tool_calls=None
            )
        )
    ],
    created=1717761150,
    model='gpt-3.5-turbo-0613',
    object='chat.completion',
    system_fingerprint=None,
    usage=CompletionUsage(completion_tokens=9, prompt_tokens=9, total_tokens=18)
)

In [26]:
messages = [
    {
        "role": "user",
        "content": "What's the weather like in Boston?",
    }
]
response = openai.chat.completions.create(
    model = "gpt-3.5-turbo-0613",
    messages = messages,
    functions = functions,
    function_call = {"name": "get_current_weather"}
)
rprint(response)

ChatCompletion(
    id='chatcmpl-9XSQwejjmUqHs46ddwM3BkWqPLPMM',
    choices=[
        Choice(
            finish_reason='stop',
            index=0,
            logprobs=None,
            message=ChatCompletionMessage(
                content=None,
                role='assistant',
                function_call=FunctionCall(
                    arguments='{\n  "location": "Boston, MA"\n}',
                    name='get_current_weather'
                ),
                tool_calls=None
            )
        )
    ],
    created=1717761150,
    model='gpt-3.5-turbo-0613',
    object='chat.completion',
    system_fingerprint=None,
    usage=CompletionUsage(completion_tokens=11, prompt_tokens=89, total_tokens=100)
)

In [27]:
messages.append(response.choices[0].message)

In [31]:
messages

[{'role': 'user', 'content': "What's the weather like in Boston?"},
 ChatCompletionMessage(content=None, role='assistant', function_call=FunctionCall(arguments='{\n  "location": "Boston, MA"\n}', name='get_current_weather'), tool_calls=None)]

In [30]:
args = json.loads(response.choices[0].message.function_call.arguments)
observation = get_current_weather(args)

Then, we can append a new message to the list contained within `messages`, representing the response of the function we just called.

We do this with a new type of message.

Notice that we pass:
- a `"role"` field specified as `"function"`, which is used **to convey to the language model that it's the response of calling a function**;
- a `"name"` field containing the function's name;
- a `"content"` field containing `observation`, e.g: the function's return value.

In [32]:
messages.append({
    "role": "function",
    "name": "get_current_weather",
    "content": observation
})

In [33]:
messages

[{'role': 'user', 'content': "What's the weather like in Boston?"},
 ChatCompletionMessage(content=None, role='assistant', function_call=FunctionCall(arguments='{\n  "location": "Boston, MA"\n}', name='get_current_weather'), tool_calls=None),
 {'role': 'function',
  'name': 'get_current_weather',
  'content': '{"location": {"location": "Boston, MA"}, "temperature": "72", "unit": "fahrenheit", "forecast": ["sunny", "windy"]}'}]

In [34]:
response = openai.chat.completions.create(
    model = "gpt-3.5-turbo-0613",
    messages = messages,
)
rprint(response)

ChatCompletion(
    id='chatcmpl-9XSZFyv9E15K313pJGKbZiK6rXRXK',
    choices=[
        Choice(
            finish_reason='stop',
            index=0,
            logprobs=None,
            message=ChatCompletionMessage(
                content='The current temperature in Boston is 72°F and it is sunny and windy.',
                role='assistant',
                function_call=None,
                tool_calls=None
            )
        )
    ],
    created=1717761665,
    model='gpt-3.5-turbo-0613',
    object='chat.completion',
    system_fingerprint=None,
    usage=CompletionUsage(completion_tokens=16, prompt_tokens=77, total_tokens=93)
)